# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 22 2023 9:37AM,263368,10,MSP221964,"Methapharm, Inc.",Released
1,May 22 2023 9:37AM,263368,10,MSP221966,"Methapharm, Inc.",Released
2,May 22 2023 9:37AM,263368,10,MSP221971,"Methapharm, Inc.",Released
3,May 22 2023 9:37AM,263368,10,MSP221976,"Methapharm, Inc.",Released
4,May 22 2023 9:37AM,263368,10,MSP221978,"Methapharm, Inc.",Released
5,May 22 2023 9:37AM,263368,10,MSP221993,"Methapharm, Inc.",Released
6,May 22 2023 9:37AM,263368,10,MSP221999,"Methapharm, Inc.",Released
7,May 22 2023 9:37AM,263368,10,MSP222000,"Methapharm, Inc.",Released
8,May 22 2023 9:35AM,263369,22,5262023,FUJIFILM Diosynth Biotechnologies Texas LLC,Released
9,May 22 2023 9:24AM,263367,10,HH-41948,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
12,263365,Released,7
13,263366,Released,4
14,263367,Released,68
15,263368,Released,8
16,263369,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
263365,NaN,7.0
263366,NaN,4.0
263367,NaN,68.0
263368,NaN,8.0
263369,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
263346,0.0,1.0
263347,1.0,0.0
263351,1.0,0.0
263353,1.0,0.0
263357,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
263346,0,1
263347,1,0
263351,1,0
263353,1,0
263357,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,263346,0,1
1,263347,1,0
2,263351,1,0
3,263353,1,0
4,263357,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,263346,,1
1,263347,1,
2,263351,1,
3,263353,1,
4,263357,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 22 2023 9:37AM,263368,10,"Methapharm, Inc."
8,May 22 2023 9:35AM,263369,22,FUJIFILM Diosynth Biotechnologies Texas LLC
9,May 22 2023 9:24AM,263367,10,Hush Hush
77,May 22 2023 9:20AM,263366,10,Hush Hush
81,May 22 2023 9:17AM,263365,10,"Methapharm, Inc."
88,May 22 2023 9:15AM,263363,16,Sartorius Stedim Filters Inc.
89,May 22 2023 9:14AM,263364,10,"Methapharm, Inc."
108,May 22 2023 9:04AM,263361,10,"Digital Brands, LLC"
128,May 22 2023 9:04AM,263362,10,Emerginnova
132,May 22 2023 9:02AM,263360,10,Alikai Health


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,May 22 2023 9:37AM,263368,10,"Methapharm, Inc.",,8
1,May 22 2023 9:35AM,263369,22,FUJIFILM Diosynth Biotechnologies Texas LLC,,1
2,May 22 2023 9:24AM,263367,10,Hush Hush,,68
3,May 22 2023 9:20AM,263366,10,Hush Hush,,4
4,May 22 2023 9:17AM,263365,10,"Methapharm, Inc.",,7
5,May 22 2023 9:15AM,263363,16,Sartorius Stedim Filters Inc.,,1
6,May 22 2023 9:14AM,263364,10,"Methapharm, Inc.",,19
7,May 22 2023 9:04AM,263361,10,"Digital Brands, LLC",,20
8,May 22 2023 9:04AM,263362,10,Emerginnova,,4
9,May 22 2023 9:02AM,263360,10,Alikai Health,7,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,May 22 2023 9:37AM,263368,10,"Methapharm, Inc.",8,
1,May 22 2023 9:35AM,263369,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1,
2,May 22 2023 9:24AM,263367,10,Hush Hush,68,
3,May 22 2023 9:20AM,263366,10,Hush Hush,4,
4,May 22 2023 9:17AM,263365,10,"Methapharm, Inc.",7,
5,May 22 2023 9:15AM,263363,16,Sartorius Stedim Filters Inc.,1,
6,May 22 2023 9:14AM,263364,10,"Methapharm, Inc.",19,
7,May 22 2023 9:04AM,263361,10,"Digital Brands, LLC",20,
8,May 22 2023 9:04AM,263362,10,Emerginnova,4,
9,May 22 2023 9:02AM,263360,10,Alikai Health,2,7


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,May 22 2023 9:37AM,263368,10,"Methapharm, Inc.",8,
1,May 22 2023 9:35AM,263369,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1,
2,May 22 2023 9:24AM,263367,10,Hush Hush,68,
3,May 22 2023 9:20AM,263366,10,Hush Hush,4,
4,May 22 2023 9:17AM,263365,10,"Methapharm, Inc.",7,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,May 22 2023 9:37AM,263368,10,"Methapharm, Inc.",8.0,NaN
1,May 22 2023 9:35AM,263369,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1.0,NaN
2,May 22 2023 9:24AM,263367,10,Hush Hush,68.0,NaN
3,May 22 2023 9:20AM,263366,10,Hush Hush,4.0,NaN
4,May 22 2023 9:17AM,263365,10,"Methapharm, Inc.",7.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,May 22 2023 9:37AM,263368,10,"Methapharm, Inc.",8.0,0.0
1,May 22 2023 9:35AM,263369,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1.0,0.0
2,May 22 2023 9:24AM,263367,10,Hush Hush,68.0,0.0
3,May 22 2023 9:20AM,263366,10,Hush Hush,4.0,0.0
4,May 22 2023 9:17AM,263365,10,"Methapharm, Inc.",7.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2633628,135.0,7.0
16,263363,1.0,0.0
19,526699,1.0,1.0
20,263347,0.0,1.0
22,526720,1.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2633628,135.0,7.0
1,16,263363,1.0,0.0
2,19,526699,1.0,1.0
3,20,263347,0.0,1.0
4,22,526720,1.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,135.0,7.0
1,16,1.0,0.0
2,19,1.0,1.0
3,20,0.0,1.0
4,22,1.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,135.0
1,16,Released,1.0
2,19,Released,1.0
3,20,Released,0.0
4,22,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,22
Status,,,,,
Executing,7.0,0.0,1.0,1.0,1.0
Released,135.0,1.0,1.0,0.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,22
0,Executing,7.0,0.0,1.0,1.0,1.0
1,Released,135.0,1.0,1.0,0.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,22
0,Executing,7.0,0.0,1.0,1.0,1.0
1,Released,135.0,1.0,1.0,0.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()